# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
S. Li  ->  S. Li  |  ['S. Li']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Samland  ->  M. Samland  |  ['M. Samland']
J. Li  ->  J. Li  |  ['J. Li']
J. Shi  ->  J. Shi  |  ['J. Shi']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
Arxiv has 65 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.17381


extracting tarball to tmp_2410.17381... done.
Retrieving document from  https://arxiv.org/e-print/2410.17455


extracting tarball to tmp_2410.17455...

 done.


...

/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{cantalloube2020wdh}Cantalloube, F., Farley, O.~J., Milli, J., Bharmal, N., Brandner, W., Correia,  C., Dohlen, K., Henning, T., Osborn, J., Por, E., et~al., ``Wind-driven halo  in high-contrast images-i. analysis of the focal-plane images of sphere,''  {\em Astronomy  Astrophysics}~{\bf 638},  A98 (2020).
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{sauvage2016}Sauvage, J.-F., Fusco, T., Lamb, M., Girard, J., Brinkmann, M., Guesalaga, A.,  Wizinowich, P., O'Neal, J., N'Diaye, M., Vigan, A., et~al., ``Tackling down  the low wind effect on sphere instrument,'' i

 done.
Retrieving document from  https://arxiv.org/e-print/2410.17999


extracting tarball to tmp_2410.17999... done.
Retrieving document from  https://arxiv.org/e-print/2410.18033
extracting tarball to tmp_2410.18033... done.


Found 67 bibliographic references in tmp_2410.18033/main_submitted_12March2024.bbl.
Retrieving document from  https://arxiv.org/e-print/2410.18078
extracting tarball to tmp_2410.18078... done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{Adams2004ApJ...611..360A}Fred~C. {Adams}, David {Hollenbach}, Gregory {Laughlin}, and Uma {Gorti}.\newblock {Photoevaporation of Circumstellar Disks Due to External  Far-Ultraviolet Radiation in Stellar Aggregates}.\newblock {\em \apj}, 611(1):360--379, August 2004.
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{Agra-Amboage2014A&A...564A..11A}V.~{Agra-Amboage}, S.~{Cabrit}, C.~{Dougados}, L.~E. {Kristensen}, L.~{Ibgui},  and J.~{Reunanen}.\newblock {Origin of the wide-angle hot H$_{2}$ in DG Tauri. New insight from  SINFONI spectro-imaging}.\newblock {\em \aap}

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.17455-b31b1b.svg)](https://arxiv.org/abs/2410.17455) | **ATOMS: ALMA three-millimeter observations of massive star-forming regions -- XVIII. On the origin and evolution of dense gas fragments in molecular shells of compact HII regions**  |
|| S. Zhang, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-10-24*|
|*Comments*| *Accepted by MNRAS, 24 pages, 14 figures*|
|**Abstract**|            Fragmentation and evolution for the molecular shells of the compact HII regions are less explored compared to their evolved counterparts. We map nine compact HII regions with a typical diameter of 0.4 pc that are surrounded by molecular shells traced by CCH. Several to a dozen dense gas fragments probed by H13CO+ are embedded in these molecular shells. These gas fragments, strongly affected by the HII region, have a higher surface density, mass, and turbulence than those outside the shells but within the same pc-scale natal clump. These features suggest that the shells swept up by the early HII regions can enhance the formation of massive dense structures that may host the birth of higher-mass stars. We examine the formation of fragments and find that fragmentation of the swept-up shell is unlikely to occur in these early HII regions, by comparing the expected time scale of shell fragmentation with the age of HII region. We propose that the appearance of gas fragments in these shells is probably the result of sweeping up pre-existing fragments into the molecular shell that has not yet fragmented. Taken together, this work provides a basis for understanding the interplay of star-forming sites with an intricate environment containing ionization feedback such as those observed in starburst regions.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.17636-b31b1b.svg)](https://arxiv.org/abs/2410.17636) | **Exoplanet Imaging Data Challenge, phase II: Comparison of algorithms in terms of characterization capabilities**  |
|| F. Cantalloube, et al. -- incl., <mark>M. Samland</mark> |
|*Appeared on*| *2024-10-24*|
|*Comments*| *Submitted to SPIE Astronomical Telescopes + Instrumentation 2024, Adaptive Optics Systems IX, Paper 13097-40*|
|**Abstract**|            In this communication, we report on the results of the second phase of the Exoplanet Imaging Data Challenge started in 2019. This second phase focuses on the characterization of point sources (exoplanet signals) within multispectral high-contrast images from ground-based telescopes. We collected eight data sets from two high-contrast integral field spectrographs (namely Gemini-S/GPI and VLT/SPHERE-IFS) that we calibrated homogeneously, and in which we injected a handful of synthetic planetary signals (ground truth) to be characterized by the data challenge participants. The tasks of the participants consist of (1) extracting the precise astrometry of each injected planetary signals, and (2) extracting the precise spectro-photometry of each injected planetary signal. Additionally, the participants may provide the 1-sigma uncertainties on their estimation for further analyses. When available, the participants can also provide the posterior distribution used to estimate the position/spectrum and uncertainties. The data are permanently available on a Zenodo repository and the participants can submit their results through the EvalAI platform. The EvalAI submission platform opened on April 2022 and closed on the 31st of May 2024. In total, we received 4 valid submissions for the astrometry estimation and 4 valid submissions for the spectrophotometry (each submission, corresponding to one pipeline, has been submitted by a unique participant). In this communication, we present an analysis and interpretation of the results.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.18033-b31b1b.svg)](https://arxiv.org/abs/2410.18033) | **JWST/NIRSpec Reveals the Nested Morphology of Disk Winds from Young Stars**  |
|| I. Pascucci, et al. -- incl., <mark>K. Schwarz</mark>, <mark>D. Semenov</mark> |
|*Appeared on*| *2024-10-24*|
|*Comments*| *This preprint has not undergone peer review or any post-submission improvements or corrections. The Version of Record of this article is published in Nature Astronomy and is available online at this https URL*|
|**Abstract**|            Radially extended disk winds could be the key to unlocking how protoplanetary disks accrete and how planets form and migrate. A distinctive characteristic is their nested morphology of velocity and chemistry. Here we report JWST/NIRSpec spectro-imaging of four young stars with edge-on disks in the Taurus star-forming region that demonstrate the ubiquity of this structure. In each source, a fast collimated jet traced by [Fe II] is nested inside a hollow cavity within wider lower-velocity H2 and, in one case, also CO ro-vibrational (v=1-0) emission. Furthermore, in one of our sources, ALMA CO(2-1) emission, paired with our NIRSpec images, reveals the nested wind structure extends further outward. This nested wind morphology strongly supports theoretical predictions for wind-driven accretion and underscores the need for theoretical work to assess the role of winds in the formation and evolution of planetary systems         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.18078-b31b1b.svg)](https://arxiv.org/abs/2410.18078) | **CO-CAVITY project: Molecular gas and star formation in void galaxies**  |
|| M. I. Rodríguez, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2024-10-24*|
|*Comments*| *26 pages, 9 figures. Accepted for publication in A&A*|
|**Abstract**|            Cosmic voids, distinguished by their low-density environment, provide a unique opportunity to explore the interplay between the cosmic environment and the processes of galaxy formation and evolution. Data on the molecular gas has been scarce so far. In this paper, we continue previous research done in the CO-CAVITY pilot project to study the molecular gas content and properties in void galaxies to search for possible differences compared to galaxies that inhabit denser structures. We observed at the IRAM 30 m telescope the CO(1-0) and CO(2-1) emission of 106 void galaxies selected from the CAVITY survey. Together with data from the literature, we obtained a sample of 200 void galaxies with CO data. We conducted a comprehensive comparison of the specific star formation rate (sSFR = SFR/M$_*$), the molecular gas fraction (MH$_2$/M$_*$), and the star formation efficiency (SFE = SFR/MH$_2$) between the void galaxies and a comparison sample of galaxies in filaments and walls, selected from the xCOLD GASS survey. We found no statistically significant difference between void galaxies and the comparison sample in the molecular gas fraction as a function of stellar mass for galaxies on the star-forming main sequence (SFMS). However, for void galaxies, the SFE was found to be constant across all stellar mass bins, while there is a decreasing trend with M$_*$ for the comparison sample. Finally, we found some indications for a smaller dynamical range in the molecular gas fraction as a function of distance to the SFMS in void galaxies. Overall, our analysis finds that the molecular gas properties of void galaxies are not very different from denser environments. The physical origin of the most significant difference that we found - a constant SFE as a function of stellar mass in void galaxies - is unclear and requires further investigation and higher-resolution data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.17590-b31b1b.svg)](https://arxiv.org/abs/2410.17590) | **HIP 8522: A Puzzling Young Solar Twin with the Lowest Detected Lithium**  |
|| J. Y. Galarza, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-10-24*|
|*Comments*| **|
|**Abstract**|            We present HIP 8522, a young solar twin with the lowest detected lithium, potentially a field blue straggler or the result of episodic early accretion. Its stellar parameters ($T_{\rm eff} = 5729 \pm 7$ K, $\log g = 4.532 \pm 0.016$ dex, $\rm{[Fe/H]} = 0.005 \pm 0.010$ dex, $v_{t} = 1.08 \pm 0.02$ km s$^{-1}$) and chemical composition were determined via spectroscopic equilibrium using high resolution spectra ($R = 60~000-165~000$). The age of HIP 8522 was estimated to be an upper limit of $<$1 Gyr through isochrone fitting and was further confirmed using chemical clocks. Spectral synthesis of the lithium line at $\sim$6707.8 Å yielded an upper lithium abundance limit of $A(\rm{Li}) <$ 0.8 dex. This value is unusually low for solar twins of similar age, which typically have $A(\rm{Li})$ values ranging from 2.0 to 3.3 dex, suggesting that $\sim$2 dex of lithium is missing. We investigate various scenarios, such as planet engulfment, sub-stellar mergers, and extra mixing. However, two distinct hypotheses provide plausible explanations for the significant depletion of lithium: one suggests that HIP 8522 is a field blue straggler formed by the merger of a close binary, while the other proposes that HIP 8522 experienced early episodic accretion. The young solar twin HIP 8522 presents an exceptional opportunity to rigorously test stellar evolution models and gain crucial insights into the internal mixing mechanisms responsible for the significant destruction of lithium.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.17833-b31b1b.svg)](https://arxiv.org/abs/2410.17833) | **Ground calibration and network of the first CATCH pathfinder**  |
|| Y. Huang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-10-24*|
|*Comments*| **|
|**Abstract**|            The Chasing All Transients Constellation Hunters (CATCH) space mission is focused on exploring the dynamic universe via X-ray follow-up observations of various transients. The first pathfinder of the CATCH mission, CATCH-1, was launched on June 22, 2024, alongside the Space-based multiband astronomical Variable Objects Monitor (SVOM) mission. CATCH-1 is equipped with narrow-field optimized Micro Pore Optics (MPOs) featuring a large effective area and incorporates four Silicon Drift Detectors (SDDs) in its focal plane. This paper presents the system calibration results conducted before the satellite integration. Utilizing the data on the performance of the mirror and detectors obtained through the system calibration, combined with simulated data, the ground calibration database can be established. Measuring the relative positions of the mirror and detector system, which were adjusted during system calibration, allows for accurate installation of the entire satellite. Furthermore, the paper outlines the operational workflow of the ground network post-satellite launch.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.17999-b31b1b.svg)](https://arxiv.org/abs/2410.17999) | **LEIA discovery of the longest-lasting and most energetic stellar X-ray flare ever detected**  |
|| X. Mao, et al. -- incl., <mark>J. Shi</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-10-24*|
|*Comments*| *submitted to ApJL, 22 pages, 9 figures, 7 tables*|
|**Abstract**|            LEIA (Lobster Eye Imager for Astronomy) detected a new X-ray transient on November 7, 2022, identified as a superflare event occurring on a nearby RS CVn-type binary HD 251108. The flux increase was also detected in follow-up observations at X-ray, UV and optical wavelengths. The flare lasted for about 40 days in soft X-ray observations, reaching a peak luminosity of ~1.1 * 10^34 erg/s in 0.5-4.0 keV, which is roughly 60 times the quiescent luminosity. Optical brightening was observed for only one night. The X-ray light curve is well described by a double "FRED" (fast rise and exponential decay) model, attributed to the cooling process of a loop arcade structure formed subsequent to the initial large loop with a half-length of ~1.9 times the radius of the host star. Time-resolved X-ray spectra were fitted with a two-temperature apec model, showing significant evolution of plasma temperature, emission measure, and metal abundance over time. The estimated energy released in the LEIA band is ~3 * 10^39 erg, suggesting this is likely the most energetic X-ray stellar flare with the longest duration detected to date.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.17381-b31b1b.svg)](https://arxiv.org/abs/2410.17381) | **Generalized $T_e$([OIII])-$T_e$(HeI) Discrepancies in Ionized Nebulae: Possible Evidence of Case B Deviations and Temperature Inhomogeneities**  |
|| J. E. Méndez-Delgado, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2024-10-24*|
|*Comments*| *Submitted to the AAS, comments and feedback from the community are welcome*|
|**Abstract**|            The physics of recombination lines (RLs) in the HeI singlet system is expected to be relatively simple, supported by accurate atomic models. We examine the intensities of HeI singlets $\lambda \lambda$3614, 3965, 5016, 6678, 7281 and the triplet HeI $\lambda$5876 in various types of ionized nebulae and compare them with theoretical predictions to test the validity of the ``Case B'' recombination scenario and the assumption of thermal homogeneity. Our analysis includes 85 spectra from Galactic and extragalactic HII regions, 90 from star-forming galaxies, and 218 planetary nebulae, all compiled by the DEep Spectra of Ionized REgions Database Extended (DESIRED-E) project. By evaluating the ratios HeI $\lambda$7281/$\lambda$6678 and HeI $\lambda$7281/$\lambda$5876, we determine $T_e$(HeI) and compare it with direct measurements of $T_e$([OIII] $\lambda$4363/$\lambda$5007). We find that $T_e$(HeI) is systematically lower than $T_e$([OIII]) across most objects and nebula types. Additionally, we identify a correlation between the abundance discrepancy factor (ADF(O$^{2+}$)) and the difference $T_e$([OIII]) - $T_e$(HeI) for planetary nebulae. We explore two potential explanations: photon loss from $n^1P \rightarrow 1^1S$ transitions and temperature inhomogeneities. Deviations from ``Case B'' may indicate photon absorption by HI rather than HeI and/or generalized ionizing photon escape, highlighting the need for detailed consideration of radiative transfer effects. If temperature inhomogeneities are widespread, identifying a common physical phenomenon affecting all ionized nebulae is crucial. Our results suggest that both scenarios can contribute to the observed discrepancies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.17455/./normal2.png', 'tmp_2410.17455/./mass_growth.png', 'tmp_2410.17455/./normal1.png']
copying  tmp_2410.17455/./normal2.png to _build/html/
copying  tmp_2410.17455/./mass_growth.png to _build/html/
copying  tmp_2410.17455/./normal1.png to _build/html/
exported in  _build/html/2410.17455.md
    + _build/html/tmp_2410.17455/./normal2.png
    + _build/html/tmp_2410.17455/./mass_growth.png
    + _build/html/tmp_2410.17455/./normal1.png
found figures ['tmp_2410.17636/./Figures_spectra_2024/Spectra_gpi1.png', 'tmp_2410.17636/./Figures_spectra_2024/Spectra_gpi2.png', 'tmp_2410.17636/./Figures_spectra_2024/Spectra_gpi3.png', 'tmp_2410.17636/./Figures_spectra_2024/Spectra_gpi4.png', 'tmp_2410.17636/./Figures_spectra_2024/Spectra_sphere1.png', 'tmp_2410.17636/./Figures_spectra_2024/Spectra_sphere2.png', 'tmp_2410.17636/./Figures_spectra_2024/Spectra_sphere3.png', 'tmp_2410.17636/./Figures_spectra_2024/Spectra_sphere4.png', 'tmp_2410.17636/./Baseline_gpi1.png', 'tmp_2

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\hi}{H~\textsc{i}}$
$\newcommand{\hii}{H~\textsc{ii}}$
$\newcommand{\msun}{\rm M_\odot}$
$\newcommand{\lsun}{\rm L_\odot}$
$\newcommand{\kms}{km~s^{-1}}$
$\newcommand{\jybeam}{Jy beam^{-1}}$
$\newcommand{\mjybeam}{mJy beam^{-1}}$
$\newcommand{\hmole}{H_2}$
$\newcommand{\cm}{cm^{-2}}$
$\newcommand{\cmcube}{cm^{-3}}$
$\newcommand{\pressureunit}{K~cm^{-3}}$
$\newcommand{\hcop}{HCO^+}$
$\newcommand{\hcopone}{HCO^+~J=1-0}$
$\newcommand{\htcop}{H^{13}CO^+}$
$\newcommand{\htcopone}{H^{13}CO^+~J=1-0}$
$\newcommand{\hcn}{HCN}$
$\newcommand{\hcnone}{HCN~J=1-0}$
$\newcommand{\htcn}{H^{13}CN}$
$\newcommand{\htcnone}{H^{13}CN~J=1-0}$
$\newcommand{\cs}{CS}$
$\newcommand{\cstwo}{CS~J=2-1}$
$\newcommand{\sio}{SiO}$
$\newcommand{\siotwo}{SiO~J=2-1}$
$\newcommand{\siofive}{SiO~J=5-4}$
$\newcommand{\hctn}{HC_3N}$
$\newcommand{\hctnele}{HC_3N~J=11-10}$
$\newcommand{\hfourtyalpha}{H40\alpha}$
$\newcommand{\chtoh}{CH_3OH}$
$\newcommand{\chtohline}{CH_3OH 2(1,1)-1(1,0)A}$
$\newcommand{çh}{CCH}$
$\newcommand{çhline}{CCH~N_{J,F}=1_{3/2,2}-0_{1/2,1}}$
$\newcommand{çhhyperfine}{CCH~N_{J,F}=1_{3/2,1}-0_{1/2,0}}$
$\newcommand{\so}{SO}$
$\newcommand{\soline}{SO v =0, 3(2)-2(1)}$
$\newcommand{\nht}{NH_3}$
$\newcommand{\mclump}{M_{\rm clump}}$
$\newcommand{\rclump}{r_{\rm clp}}$
$\newcommand{\RGC}{R_{\rm GC}}$
$\newcommand{\mvir}{M_{\rm vir}}$
$\newcommand{\rvir}{R_{\rm vir}}$
$\newcommand{\mcore}{M_{\rm core}}$
$\newcommand{\sufdenscore}{\Sigma_{\rm core}}$
$\newcommand{\rcoreeff}{r_{\rm core,eff}}$
$\newcommand{\tkin}{T_{\rm kin}}$
$\newcommand{\tex}{T_{\rm ex}}$
$\newcommand{\tbg}{T_{\rm bg}}$
$\newcommand{\tr}{T_{\rm r}}$
$\newcommand{\te}{T_{\rm e}}$
$\newcommand{\pressurei}{P_{\rm i}}$
$\newcommand{\RecRate}{\dot{N}_{\rm rec}}$
$\newcommand{\IonRate}{\dot{N}_{\rm ion}}$
$\newcommand{\nelectron}{n_{\rm e}}$
$\newcommand{\tdyn}{t_{\rm dyn}}$
$\newcommand{\ci}{c_{\rm i}}$
$\newcommand{\nini}{n_{\rm ini}}$
$\newcommand{\ahfourtya}{A_{\rm H40\alpha}}$
$\newcommand{\lhfourtya}{L_{\rm H40\alpha}}$
$\newcommand{\fhfourtya}{\nu_{\rm H40\alpha}}$
$\newcommand{\rhfourtya}{r_{\rm H40\alpha,eff}}$
$\newcommand{\hcopabundance}{\chi(\hcop)}$
$\newcommand{\hncabundance}{\chi(\hnc)}$
$\newcommand{\nthpabundance}{\chi(\nthp)}$
$\newcommand{\nthphcopabundance}{\chi(\hcop)/\chi(\nthp)}$
$\newcommand{\nthphncabundance}{\chi(\hnc)/\chi(\nthp)}$
$\newcommand{\tcoab}{\chi(^{13}CO)}$
$\newcommand{\ceoab}{\chi(C^{18}O)}$
$\newcommand{\htcopab}{\chi_{\rm H^{13}CO^+}}$
$\newcommand{\co}{CO}$
$\newcommand{\tco}{^{13}CO}$
$\newcommand{\ceo}{C^{18}O}$
$\newcommand{\coone}{^{12}CO J=1-0}$
$\newcommand{\cotwo}{^{12}CO J=2-1}$
$\newcommand{\cothr}{^{12}CO J=3-2}$
$\newcommand{\tcoone}{^{13}CO J=1-0}$
$\newcommand{\tcotwo}{^{13}CO J=2-1}$
$\newcommand{\tcothr}{^{13}CO J=3-2}$
$\newcommand{\ceoone}{C^{18}O~J=1-0}$
$\newcommand{\ceotwo}{C^{18}O J=2-1}$
$\newcommand{\ceothr}{C^{18}O J=3-2}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\nhtcd}{N_{\rm H_{2}}}$
$\newcommand{\dustt}{T_{\rm dust}}$
$\newcommand{\nhtnd}{n_{\rm H_{2}}}$
$\newcommand{\nhtcdpeak}{N_{\rm H_{2}}^{\rm peak}}$
$\newcommand{\ncchcd}{N_{\rm CCH}}$
$\newcommand{\nnhtcd}{N_{\rm NH_{3}}}$
$\newcommand{\pnhtcd}{p_{N_{\rm H_{2}}}}$
$\newcommand{\pdustt}{p_{T_{\rm dust}}}$
$\newcommand{\vlsr}{\rm v_{lsr}}$
$\newcommand{\vinfall}{\rm v_{infall}}$
$\newcommand{\mjth}{M_{\rm J}^{\rm th}}$
$\newcommand{\lambdajth}{\lambda_{\rm J}^{\rm th}}$
$\newcommand{\mjthclump}{M_{\rm J, clp}^{\rm th}}$
$\newcommand{\lambdajthclump}{\lambda_{\rm J, clp}^{\rm th}}$
$\newcommand{\mjtotclump}{M_{\rm J, clp}^{\rm tot}}$
$\newcommand{\lambdajtotclump}{\lambda_{\rm J, clp}^{\rm tot}}$
$\newcommand{\mjflowclump}{M_{\rm J, clp}^{\rm com, flow}}$
$\newcommand{\lambdajflowclump}{\lambda_{\rm J, clp}^{\rm com, flow}}$
$\newcommand{\mjthens}{M_{\rm J, ens}^{\rm th}}$
$\newcommand{\lambdajthens}{\lambda_{\rm J, ens}^{\rm th}}$
$\newcommand{\Pex}{P_{\rm ex}}$
$\newcommand{\Pin}{P_{\rm in}}$
$\newcommand{\mach}{\mathcal{M}}$
$\newcommand{\mjtu}{M_{\rm J}^{\rm tu}}$
$\newcommand{\lambdajtu}{\lambda_{\rm J}^{\rm tu}}$
$\newcommand{\av}{A_{\rm v}}$
$\newcommand$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# ATOMS: ALMA three-millimeter observations of massive star-forming regions -- XVIII. On the origin and evolution of dense gas fragments in molecular shells of compact $\hii$ regions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.17455-b31b1b.svg)](https://arxiv.org/abs/2410.17455)<mark>Appeared on: 2024-10-24</mark> -  _Accepted by MNRAS, 24 pages, 14 figures_

</div>
<div id="authors">

S. Zhang, et al. -- incl., <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** Fragmentation and evolution for the molecular shells of the compact $\hii$ regions are less explored compared to their evolved counterparts. We map nine compact $\hii$ regions with a typical diameter of 0.4 pc that are surrounded by molecular shells traced by çh. Several to a dozen dense gas fragments probed by $\htcop$ are embedded in these molecular shells. These gas fragments, strongly affected by the $\hii$ region, have a higher surface density, mass, and turbulence than those outside the shells but within the same pc-scale natal clump. These features suggest that the shells swept up by the early $\hii$ regions can enhance the formation of massive dense structures that may host the birth of higher-mass stars.  We examine the formation of fragments and find that fragmentation of the swept-up shell is unlikely to occur in these early $\hii$ regions, by comparing the expected time scale of shell fragmentation with the age of $\hii$ region. We propose that the appearance of gas fragments in these shells is probably the result of sweeping up pre-existing fragments into the molecular shell that has not yet fragmented. Taken together, this work provides a basis for understanding the interplay of star-forming sites with an intricate environment containing ionization feedback such as those observed in starburst regions.

</div>

<div id="div_fig1">

<img src="tmp_2410.17455/./normal2.png" alt="Fig6" width="100%"/>

**Figure 6. -** $\htcop$one emission and extracted fragments. The color images show the $\htcop$one moment 0 maps (PB corrected) produced by the technique described in $\ci$tet{Dame2011} with a channel clipping level of $4\times\rm rms$. The red and cyan contours outline the extracted fragments close to and far from the central $\hi$i regions (N and F fragments; see Section \ref{SECTION:INFLUENCE}), respectively. Pink contours indicate the edges of $\hi$i regions. Magenta circles indicate the peaks of angular two-point correlation function (solid: first peak, dashed: second peak) discussed in Section \ref{SUBSECTION:FRAGMENTATION-DISTRIBUTION}. The lime lines are the $\tex$ttt{minimum spanning tree (MST)} connections for the N fragments probably located in shell. The two black circles indicate the 20\% and 30\% power points of PB. (*FIGURE:NORMAL2*)

</div>
<div id="div_fig2">

<img src="tmp_2410.17455/./mass_growth.png" alt="Fig4" width="100%"/>

**Figure 4. -** Fragment mass, fragment-to-clump mass ratio, and their relations with radius of corresponding $\hi$i regions. The distance to the Earth for each region is indicated by gray dashed lines (corresponding to right y-axis) in panels (a) and (b). Panel (a) The red and blue lines indicate the variation of N and F fragments median mass with radius of $\hi$i region, respectively. Dots only show the N fragments. Panel (b) Mass ratio of total N fragments to natal clump (black lines and triangles). The numbers indicate the corresponding values of mass ratio.  (*FIGURE:MASS_GROWTH*)

</div>
<div id="div_fig3">

<img src="tmp_2410.17455/./normal1.png" alt="Fig5" width="100%"/>

**Figure 5. -** Compact $\hi$i regions in this work. The color images display the $\hfourtyalpha$ moment 0 maps created using the technique described in $\ci$tet{Dame2011}, with a channel clipping level of $4\times\rm rms$. The black solid contours and the gray dashed contours represent the çh emission and the ATLASGAL 870 \micron emission $\ci$tep{Schuller2009}, respectively. The contour levels start from 20\% of the maximum to the maximum with eight evenly spaced steps on a linear scale and on a logarithmic scale, respectively. The yellow contours and the red circles indicate the edges of $\hi$i regions and their corresponding circular sizes $r_{\rm H40\alpha,eff}$, as described in Section \ref{SECTION:HII-REGION}. All moment 0 maps shown here have been corrected by primary beam (PB). Two black circles indicate the 20\% and 30\% power points of PB, respectively. Note that the çhline line of I15570$-$5227 and I17160$-$3707 is located on the border of the spectral window and therefore only a part of the çhline emission channels are actually integrated. (*FIGURE:NORMAL1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.17455"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bad}[1]{\textcolor{BrickRed}{#1}}$
$\newcommand{\nom}[1]{\textcolor{YellowOrange}{#1}}$
$\newcommand{\good}[1]{\textcolor{PineGreen}{#1}}$
$\newcommand{\baselinestretch}{1.0}$</div>



<div id="title">

# Exoplanet Imaging Data Challenge, phase II: Comparison of algorithms in terms of characterization capabilities

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.17636-b31b1b.svg)](https://arxiv.org/abs/2410.17636)<mark>Appeared on: 2024-10-24</mark> -  _Submitted to SPIE Astronomical Telescopes + Instrumentation 2024, Adaptive Optics Systems IX, Paper 13097-40_

</div>
<div id="authors">

Cantalloube~F., et al.

</div>
<div id="abstract">

**Abstract:** In this communication, we report on the results of the second phase of the $*Exoplanet Imaging Data Challenge*$ started in 2019. This second phase focuses on the characterization of point sources (exoplanet signals) within multispectral high-contrast images from ground-based telescopes. We collected eight data sets from two high-contrast integral field spectrographs (namely Gemini-S/GPI and VLT/SPHERE-IFS) that we calibrated homogeneously and in which we injected a handful of synthetic planetary signals (ground truth) to be characterized by the data challenge participants.The tasks of the participants consist of (1) extracting the precise astrometry of each injected planetary signals, and (2) extracting the precise spectro-photometry of each injected planetary signal. Additionally, the participants may provide the 1-sigma uncertainties on their estimation for further analyses. When available, the participants can also provide the posterior distribution used to estimate the position/spectrum and uncertainties.The data are permanently available on a Zenodo repository and the participants can submit their results through the EvalAI platform.The EvalAI submission platform opened on April 2022 and closed on the 31st of May 2024.In total, we received $4$ valid submissions for the astrometry estimation and $4$ valid submissions for the spectrophotometry (each submission, corresponding to one pipeline, has been submitted by a unique participant).In this communication, we present an analysis and interpretation of the results.

</div>

<div id="div_fig1">

<img src="tmp_2410.17636/./Figures_spectra_2024/Spectra_gpi1.png" alt="Fig7.1" width="25%"/><img src="tmp_2410.17636/./Figures_spectra_2024/Spectra_gpi2.png" alt="Fig7.2" width="25%"/><img src="tmp_2410.17636/./Figures_spectra_2024/Spectra_gpi3.png" alt="Fig7.3" width="25%"/><img src="tmp_2410.17636/./Figures_spectra_2024/Spectra_gpi4.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** GPI estimated spectra for each injected planetary signal (in contrast to the star). The ground-truth is shown as a black solid line. The top panel shows the estimated spectra by each method, while the bottom panel highlights the difference between the ground-truth spectrum and the estimated spectra.
    For reference, the spectra extracted with the baseline method are shown with red solid lines (PCA-SADI with NEGFC). The spectra extracted with the three submissions are shown in gold (pyKLIP-FM), green (RSM), and blue (AMAT).
    From top to bottom: data set _gpi1_, _gpi2_, _gpi3_, and _gpi4_. The wavelength range of GPI is $[1.495-1.797]\mathrm{\mu m}$(around the H-band), shared in 37 spectral channels. (*fig:photom_gpi*)

</div>
<div id="div_fig2">

<img src="tmp_2410.17636/./Figures_spectra_2024/Spectra_sphere1.png" alt="Fig8.1" width="25%"/><img src="tmp_2410.17636/./Figures_spectra_2024/Spectra_sphere2.png" alt="Fig8.2" width="25%"/><img src="tmp_2410.17636/./Figures_spectra_2024/Spectra_sphere3.png" alt="Fig8.3" width="25%"/><img src="tmp_2410.17636/./Figures_spectra_2024/Spectra_sphere4.png" alt="Fig8.4" width="25%"/>

**Figure 8. -** SPHERE-IFS estimated spectra for each injected planetary signal (in contrast to the star). The ground-truth is shown as a black solid line. The top panel shows the estimated spectra by each method, while the bottom panel highlights the difference between the ground-truth spectrum and the estimated spectra.
    For reference, the spectra extracted with the baseline method are shown with red solid lines (PCA-SADI with NEGFC). The spectra extracted with the three submissions are shown in gold (pyKLIP-FM), green (RSM), and blue (AMAT). From top to bottom: data set _sphere1_, _sphere2_, _sphere3_, and _sphere4_. The wavelength range of SPHERE-IFS is $[0.957-1.329]\mathrm{\mu m}$(YJ-bands), shared in 39 spectral channels. (*fig:photom_sphere*)

</div>
<div id="div_fig3">

<img src="tmp_2410.17636/./Baseline_gpi1.png" alt="Fig1.1" width="25%"/><img src="tmp_2410.17636/./Baseline_gpi2.png" alt="Fig1.2" width="25%"/><img src="tmp_2410.17636/./Baseline_gpi3.png" alt="Fig1.3" width="25%"/><img src="tmp_2410.17636/./Baseline_gpi4.png" alt="Fig1.4" width="25%"/>

**Figure 1. -** Astrometry: GPI data baseline results for each injection. On each image, the location of the injected synthetic planetary signals are shown with the orange, red and purple circles. From left to right: Temporal  median image at the shortest wavelength (left); residual map after applying a PCA-SADI post-processing, as implemented in the VIP package and using the first 10 principal components to build the reference image for subtraction (middle-left); corresponding SNR map, as implemented in the VIP package (middle-right); and astrometry estimations for each injections, using the baseline NEGFC technique (right). The latter highlights the relative astrometry errors (in pixels) for each injection (color coded) compared to the ground-truth shown as a black diamond in the center of the image. The shaded area correspond to the 1-sigma uncertainty on the astrometry estimation. From top to bottom: data set _gpi1_, _gpi2_, _gpi3_, and _gpi4_. (*fig:data_gpi*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.17636"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\actaa}{Acta Astron.}$
$\newcommand{\araa}{Annu. Rev. Astron. Astrophys.}$
$\newcommand{\areps}{Annu. Rev. Earth Planet. Sci.}$
$\newcommand{\aar}{Astron. Astrophys. Rev.}$
$\newcommand{\ab}{Astrobiology}$
$\newcommand{\aj}{Astron. J.}$
$\newcommand{\ac}{Astron. Comput.}$
$\newcommand{\apart}{Astropart. Phys.}$
$\newcommand{\apj}{Astrophys. J.}$
$\newcommand{\apjl}{Astrophys. J. Lett.}$
$\newcommand{\apjs}{Astrophys. J. Suppl. Ser.}$
$\newcommand{\ao}{Appl. Opt.}$
$\newcommand{\apss}{Astrophys. Space Sci.}$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand{\aapr}{Astron. Astrophys. Rev.}$
$\newcommand{\aaps}{Astron. Astrophys. Suppl.}$
$\newcommand{\baas}{Bull. Am. Astron. Soc.}$
$\newcommand{\caa}{Chin. Astron. Astrophys.}$
$\newcommand{\cjaa}{Chin. J. Astron. Astrophys.}$
$\newcommand{\cqg}{Class. Quantum Gravity}$
$\newcommand{\epsl}{Earth Planet. Sci. Lett.}$
$\newcommand{\frass}{Front. Astron. Space Sci.}$
$\newcommand{\gal}{Galaxies}$
$\newcommand{\gca}{Geochim. Cosmochim. Acta}$
$\newcommand{\grl}{Geophys. Res. Lett.}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\ija}{Int. J. Astrobiol.}$
$\newcommand{\jatis}{J. Astron. Telesc. Instrum. Syst.}$
$\newcommand{\jcap}{J. Cosmol. Astropart. Phys.}$
$\newcommand{\jgr}{J. Geophys. Res.}$
$\newcommand{\jgrp}{J. Geophys. Res.: Planets}$
$\newcommand{\jqsrt}{J. Quant. Spectrosc. Radiat. Transf.}$
$\newcommand{\lrca}{Living Rev. Comput. Astrophys.}$
$\newcommand{\lrr}{Living Rev. Relativ.}$
$\newcommand{\lrsp}{Living Rev. Sol. Phys.}$
$\newcommand{\memsai}{Mem. Soc. Astron. Italiana}$
$\newcommand{\maps}{Meteorit. Planet. Sci.}$
$\newcommand{\mnras}{Mon. Not. R. Astron. Soc.}$
$\newcommand{\nat}{Nature}$
$\newcommand{\nastro}{Nat. Astron.}$
$\newcommand{\ncomms}{Nat. Commun.}$
$\newcommand{\ngeo}{Nat. Geosci.}$
$\newcommand{\nphys}{Nat. Phys.}$
$\newcommand{\na}{New Astron.}$
$\newcommand{\nar}{New Astron. Rev.}$
$\newcommand{\physrep}{Phys. Rep.}$
$\newcommand{\pra}{Phys. Rev. A}$
$\newcommand{\prb}{Phys. Rev. B}$
$\newcommand{\prc}{Phys. Rev. C}$
$\newcommand{\prd}{Phys. Rev. D}$
$\newcommand{\pre}{Phys. Rev. E}$
$\newcommand{\prl}{Phys. Rev. Lett.}$
$\newcommand{\psj}{Planet. Sci. J.}$
$\newcommand{\planss}{Planet. Space Sci.}$
$\newcommand{\pnas}{Proc. Natl Acad. Sci. USA}$
$\newcommand{\procspie}{Proc. SPIE}$
$\newcommand{\pasa}{Publ. Astron. Soc. Aust.}$
$\newcommand{\pasj}{Publ. Astron. Soc. Jpn}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pac.}$
$\newcommand{\raa}{Res. Astron. Astrophys.}$
$\newcommand{\rmxaa}{Rev. Mexicana Astron. Astrofis.}$
$\newcommand{\sci}{Science}$
$\newcommand{\sciadv}{Sci. Adv.}$
$\newcommand{\solphys}{Sol. Phys.}$
$\newcommand{\sovast}{Soviet Astron.}$
$\newcommand{\ssr}{Space Sci. Rev.}$
$\newcommand{\uni}{Universe}$
$\newcommand{\micron}{\mum}$
$\newcommand{\oi}{[O {\scriptsize I}]}$
$\newcommand{\feii}{[Fe {\scriptsize II}]}$
$\newcommand{\sii}{[S {\scriptsize II}]}$
$\newcommand{\neii}{[Ne {\scriptsize II}]}$
$\newcommand{\ilaria}[1]$
$\newcommand{\suzan}[1]$
$\newcommand{\tracy}[1]$
$\newcommand{\sylvie}[1]$</div>



<div id="title">

# JWST/NIRSpec Reveals the Nested Morphology of Disk Winds from Young Stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.18033-b31b1b.svg)](https://arxiv.org/abs/2410.18033)<mark>Appeared on: 2024-10-24</mark> -  _This preprint has not undergone peer review or any post-submission improvements or corrections. The Version of Record of this article is published in Nature Astronomy and is available online at this https URL_

</div>
<div id="authors">

I. Pascucci, et al. -- incl., <mark>K. Schwarz</mark>, <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** Radially extended disk winds could be the key to unlocking how protoplanetary disks accrete and how planets form and migrate. A distinctive characteristic is their nested morphology of velocity and chemistry. Here we report JWST/NIRSpec spectro-imaging of four young stars with edge-on disks in the Taurus star-forming regionthat demonstrate the ubiquity of this structure. In each source, a fast collimated  jet traced by $\feii$ is nested inside a hollow cavity within wider lower-velocity $H_2$ and, in one case, also CO ro-vibrational (v=1-0) emission. Furthermore, in one of our sources, ALMA CO (2-1) emission, paired with our NIRSpec images, reveals the nested wind structure extends further outward.This nested wind morphology strongly supports theoretical predictions for wind-driven accretion and underscores the need for theoretical work to assess the role of winds in the formation and evolution of planetary systems. $_ In accordance with the Springer Nature publishing agreement, we can only post the submitted version of our article, prior to peer review. The version of record is published here: https://doi.org/10.1038/s41550-024-02385-7_$

</div>

<div id="div_fig1">

<img src="tmp_2410.18033/./FSTauB_rgb_composite.png" alt="Fig2.1" width="25%"/><img src="tmp_2410.18033/./HH30_rgb_composite.png" alt="Fig2.2" width="25%"/><img src="tmp_2410.18033/./IRAS04302_rgb_composite.png" alt="Fig2.3" width="25%"/><img src="tmp_2410.18033/./TAU042021_rgb_composite.png" alt="Fig2.4" width="25%"/>

**Figure 2. -** Composite images NIRSpec: blue $\feii$  emission at 1.644 $\micron$  – green $H_2$ emission at 4.695 $\micron$ - red continuum emission at the $H_2$ line. CO emission from $\sim 4.7-4.9$\micron is shown in the right bottom sub-panel in gray – the 50\% of the peak emission contour (and for HH 30 also the 30\%) is  shown in gray in the composite image. In every instance, the collimated jet traced in $\feii$ is nestled inside the wider $H_2$ emission and, in the case of HH 30, also the CO (v=1-0) emission. (*fig:composite*)

</div>
<div id="div_fig2">

<img src="tmp_2410.18033/./fig_jets_winds.png" alt="Fig3" width="100%"/>

**Figure 3. -** Left panel: FWHM of the jet's blueshifted component as a function of distance from the disk plane. Right panel: Edges of the wind's blueshifted component vs. the disk's plane. In both panels filled symbols represent data points used in the linear fit to estimate the jet and the wind semi-opening angles and the geometric radii (see Table \ref{tab:jetwindprop}). Best-fits are shown with dashed lines. Data points near the disk plane are not included in the fit due to inadequate sampling of the FWHM and scattering (see Sect. \ref{sect:methods} for details). For IRAS 043202 we exclude points above 1.2$"$ as they clearly deviate from a wind-like morphology. HH 30 has the narrowest jet and wind in our sample.  (*fig:jets_winds*)

</div>
<div id="div_fig3">

<img src="tmp_2410.18033/./fig_1dspk.png" alt="Fig1" width="100%"/>

**Figure 1. -** Spectra integrated over the NIRSpec IFU. A few of the strongest lines are marked with gray dotted lines. All of these  lines are spatially extended. The transitions analyzed in this study are highlighted in boldface. (*fig:lines*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.18033"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

228  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
